# Bare critical masses of Uranium
## Various approximations of composition
Hitoshi Cullinan 06 NOV 2022

In [1]:
import openmc
import openmc.model
import numpy as np
from math import pi
import matplotlib.pyplot as plt
%matplotlib inline

## Define Materials of interest

In [23]:
# Highly enriched uranium 93.15% U-235
# U.S. Average taken from PNNL Material compendium
HEUden = 18.724760
HEU = openmc.Material(name="HEU")
HEU.set_density('g/cc', 18.724760)
HEU.add_nuclide('U234', 0.009849, 'ao')
HEU.add_nuclide('U235', 0.931550, 'ao')
HEU.add_nuclide('U236', 0.004500, 'ao')
HEU.add_nuclide('U238', 0.054150, 'ao')

# HEU 234/236 approximation
# approximation guessing the 234/236 content based on 235 enrichment
HEU2426den = 18.724760
HEU2426 = openmc.Material(name="HEU 24/26")
HEU2426.set_density('g/cc', 18.724760)
HEU2426.add_nuclide('U234', 0.009849, 'ao')
HEU2426.add_nuclide('U235', 0.931550, 'ao')
HEU2426.add_nuclide('U236', 0.004500, 'ao')
HEU2426.add_nuclide('U238', 0.054150, 'ao')

# HEU 234 only approximation
# approximation guessing the 234 content based on 235 enrichment
# 236 neglected and added to 238 total
HEU24den = 18.724760
HEU24 = openmc.Material(name="HEU 24")
HEU24.set_density('g/cc', 18.724760)
HEU24.add_nuclide('U234', 0.009849, 'ao')
HEU24.add_nuclide('U235', 0.931550, 'ao')
HEU24.add_nuclide('U238', 0.058650, 'ao')

# HEU di-isotopic rough approx
# approximation of compostition using only U-235 and U-238
diHEUden = 18.724760
diHEU = openmc.Material(name="diHEU")
diHEU.set_density('g/cc', 18.724760)
diHEU.add_nuclide('U235', 0.931550, 'ao')
diHEU.add_nuclide('U238', 0.068450, 'ao')

# HEU openmc enrichment tool approximation
# approximation of compostition using openmc enrichment tool
omcHEUden = 18.724760
omcHEU = openmc.Material(name="omcHEU")
omcHEU.set_density('g/cc', 18.724760)
omcHEU.add_element('U',1, enrichment=93.155)

# High Assay Low Enriched Uranoum
# weight percents taken from INL UO2 HALEU paper
HALEUden = 18.944386 # approximated with LEU density
HALEU = openmc.Material(name="HALEU")
HALEU.set_density('g/cc', HALEUden)
HALEU.add_nuclide('U234', 0.0017, 'wo')
HALEU.add_nuclide('U235', 0.1939, 'wo')
HALEU.add_nuclide('U236', 0.0058, 'wo')
HALEU.add_nuclide('U238', 0.7986, 'wo')

# HEU 234/236 approximation
# approximation guessing the 234/236 content based on 235 enrichment
HALEU2426den = 18.944386
enrich = 0.195875
HALEU2426 = openmc.Material(name="HALEU 24/26")
HALEU2426.set_density('g/cc', HEU2426den)
HALEU2426.add_nuclide('U234', enrich*0.009, 'ao')
HALEU2426.add_nuclide('U235', enrich, 'ao')
HALEU2426.add_nuclide('U236', enrich*0.0045, 'ao')
HALEU2426.add_nuclide('U238', enrich*4.0917, 'ao')

# HEU 234 only approximation
# approximation guessing the 234 content based on 235 enrichment
# 236 neglected and added to 238 total
HALEU24den = 18.944386
HALEU24 = openmc.Material(name="HALEU 24")
HALEU24.set_density('g/cc', HALEU24den)
HALEU24.add_nuclide('U234', enrich*0.009, 'ao')
HALEU24.add_nuclide('U235', enrich, 'ao')
HALEU24.add_nuclide('U238', enrich*4.0962, 'ao')

# HEU di-isotopic rough approx
# approximation of compostition using only U-235 and U-238
diHALEUden = 18.944386
diHALEU = openmc.Material(name="diHALEU")
diHALEU.set_density('g/cc', diHALEUden)
diHALEU.add_nuclide('U235', enrich, 'ao')
diHALEU.add_nuclide('U238', 1-enrich, 'ao')

# HEU openmc enrichment tool approximation
# approximation of compostition using openmc enrichment tool
omcHALEUden = 18.944386
omcHALEU = openmc.Material(name="omcHALEU")
omcHALEU.set_density('g/cc', omcHALEUden)
omcHALEU.add_element('U',1, enrichment=enrich*100)




/home/m231326/anaconda3/envs/omc/lib/python3.10/site-packages/openmc/material.py:640: UserWarning: A uranium enrichment of 93.155 was given for Material ID="20". OpenMC assumes the U234/U235 mass ratio is constant at 0.008, which is only valid at low enrichments. Consider setting the isotopic composition manually for enrichments over 5%.
  warnings.warn(msg)
/home/m231326/anaconda3/envs/omc/lib/python3.10/site-packages/openmc/material.py:640: UserWarning: A uranium enrichment of 19.5875 was given for Material ID="25". OpenMC assumes the U234/U235 mass ratio is constant at 0.008, which is only valid at low enrichments. Consider setting the isotopic composition manually for enrichments over 5%.
  warnings.warn(msg)


## Create Function for  radius search
make sphere a fuction of radius (and material ?)

In [3]:
def build_model(radius, fuel):
    
    
    materials = openmc.Materials([fuel])
    
    # create sphere with radius parameter
    sphere_radius = openmc.Sphere(x0=0,y0=0,z0=0,r=radius, boundary_type='vacuum', name='BCM')
    
    # create core cell
    core_cell = openmc.Cell(name='Bare Critical Sphere')
    core_cell.fill = fuel
    core_cell.region = -sphere_radius
    
    # create universe geometry
    root_universe = openmc.Universe(name='root universe')
    root_universe.add_cells([core_cell])
    
    geometry = openmc.Geometry(root_universe)
    # define criticality settings
    settings = openmc.Settings()
    settings.run_mode = 'eigenvalue' # keff calculation
    settings.particles = 5000 # particles per batch (mo betta)
    settings.batches = 250 # number of batches
    settings.inactive = 50 # inactive batches
    
    settings.output = {'tallies': False}
    
    model = openmc.model.Model(geometry,materials,settings)
    
    return model

### Conduct Radius Search of 

In [4]:
crit_r, guesses, keffs = openmc.search_for_keff(build_model, bracket=[1,20],model_args={'fuel':HEU},
                                                tol=1e-4, print_iterations=True,
                                               run_args={'output':False})
# print results and collect data
print('PNNL HEU')
print('The bare critical sphere radius is %7.4f cm.' % crit_r)
crit_v = 4/3*pi*crit_r**3 # volume of critical sphere (cc)

BCM = crit_v * HEUden /1000 # mass of critical radius (kg)
print('The bare critical mass is %7.3f kg.' % BCM)

BCMs = np.array(BCM)
print(BCMs,
      '\n')

Iteration: 1; Guess of 1.00e+00 produced a keff of 0.11862 +/- 0.00009
Iteration: 2; Guess of 2.00e+01 produced a keff of 1.69311 +/- 0.00094
Iteration: 3; Guess of 1.05e+01 produced a keff of 1.15371 +/- 0.00073
Iteration: 4; Guess of 5.75e+00 produced a keff of 0.68532 +/- 0.00048
Iteration: 5; Guess of 8.12e+00 produced a keff of 0.93849 +/- 0.00060
Iteration: 6; Guess of 9.31e+00 produced a keff of 1.04954 +/- 0.00070
Iteration: 7; Guess of 8.72e+00 produced a keff of 0.99355 +/- 0.00070
Iteration: 8; Guess of 9.02e+00 produced a keff of 1.02300 +/- 0.00066
Iteration: 9; Guess of 8.87e+00 produced a keff of 1.00738 +/- 0.00064
Iteration: 10; Guess of 8.79e+00 produced a keff of 1.00071 +/- 0.00060
Iteration: 11; Guess of 8.76e+00 produced a keff of 0.99818 +/- 0.00067
Iteration: 12; Guess of 8.77e+00 produced a keff of 0.99969 +/- 0.00070
Iteration: 13; Guess of 8.78e+00 produced a keff of 1.00049 +/- 0.00072
Iteration: 14; Guess of 8.78e+00 produced a keff of 0.99993 +/- 0.00065
I

In [5]:
crit_r, guesses, keffs = openmc.search_for_keff(build_model, bracket=[1,20],
                                                model_args={'fuel':HEU24},
                                                tol=1e-4, print_iterations=True,
                                               run_args={'output':False})

print('HEU no 236 approximation')
print('The bare critical sphere radius is %7.4f cm.' % crit_r)
crit_v = 4/3*pi*crit_r**3 # volume of critical sphere (cc)

BCM = crit_v * HEUden /1000 # mass of critical radius (kg)
print('The bare critical mass is %7.3f kg.' % BCM)

BCMs = np.append(BCMs,[BCM])
print(BCMs)

Iteration: 1; Guess of 1.00e+00 produced a keff of 0.11849 +/- 0.00009
Iteration: 2; Guess of 2.00e+01 produced a keff of 1.69631 +/- 0.00091
Iteration: 3; Guess of 1.05e+01 produced a keff of 1.15228 +/- 0.00077
Iteration: 4; Guess of 5.75e+00 produced a keff of 0.68433 +/- 0.00051
Iteration: 5; Guess of 8.12e+00 produced a keff of 0.93618 +/- 0.00065
Iteration: 6; Guess of 9.31e+00 produced a keff of 1.04814 +/- 0.00069
Iteration: 7; Guess of 8.72e+00 produced a keff of 0.99432 +/- 0.00063
Iteration: 8; Guess of 9.02e+00 produced a keff of 1.02121 +/- 0.00072
Iteration: 9; Guess of 8.87e+00 produced a keff of 1.00736 +/- 0.00069
Iteration: 10; Guess of 8.79e+00 produced a keff of 1.00173 +/- 0.00065
Iteration: 11; Guess of 8.76e+00 produced a keff of 0.99704 +/- 0.00071
Iteration: 12; Guess of 8.77e+00 produced a keff of 0.99886 +/- 0.00070
Iteration: 13; Guess of 8.78e+00 produced a keff of 1.00003 +/- 0.00066
Iteration: 14; Guess of 8.78e+00 produced a keff of 1.00011 +/- 0.00065
I

In [6]:
crit_r, guesses, keffs = openmc.search_for_keff(build_model, bracket=[1,50],
                                                model_args={'fuel':diHEU},
                                                tol=1e-4, print_iterations=True,
                                               run_args={'output':False})

print('Di-isotopic HEU')
print('The bare critical sphere radius is %7.4f cm.' % crit_r)
crit_v = 4/3*pi*crit_r**3 # volume of critical sphere (cc)

BCM = crit_v * HEUden /1000 # mass of critical radius (kg)
print('The bare critical mass is %7.3f kg.' % BCM)

BCMs = np.append(BCMs,[BCM])
print(BCMs)

Iteration: 1; Guess of 1.00e+00 produced a keff of 0.11791 +/- 0.00009
Iteration: 2; Guess of 5.00e+01 produced a keff of 2.12174 +/- 0.00071
Iteration: 3; Guess of 2.55e+01 produced a keff of 1.85058 +/- 0.00085
Iteration: 4; Guess of 1.32e+01 produced a keff of 1.35328 +/- 0.00086
Iteration: 5; Guess of 7.12e+00 produced a keff of 0.83133 +/- 0.00056
Iteration: 6; Guess of 1.02e+01 produced a keff of 1.12153 +/- 0.00074
Iteration: 7; Guess of 8.66e+00 produced a keff of 0.98460 +/- 0.00070
Iteration: 8; Guess of 9.42e+00 produced a keff of 1.05493 +/- 0.00070
Iteration: 9; Guess of 9.04e+00 produced a keff of 1.01967 +/- 0.00075
Iteration: 10; Guess of 8.85e+00 produced a keff of 1.00145 +/- 0.00065
Iteration: 11; Guess of 8.75e+00 produced a keff of 0.99399 +/- 0.00068
Iteration: 12; Guess of 8.80e+00 produced a keff of 0.99805 +/- 0.00069
Iteration: 13; Guess of 8.82e+00 produced a keff of 0.99934 +/- 0.00070
Iteration: 14; Guess of 8.84e+00 produced a keff of 1.00089 +/- 0.00075
I

In [7]:
crit_r, guesses, keffs = openmc.search_for_keff(build_model, bracket=[1,50],
                                                model_args={'fuel':omcHEU},
                                                tol=1e-4, print_iterations=True,
                                               run_args={'output':False})

print('omc HEU Approximation')
print('The bare critical sphere radius is %7.4f cm.' % crit_r)
crit_v = 4/3*pi*crit_r**3 # volume of critical sphere (cc)

BCM = crit_v * HEUden /1000 # mass of critical radius (kg)
print('The bare critical mass is %7.3f kg.' % BCM)

BCMs = np.append(BCMs,[BCM])
print(BCMs)

Iteration: 1; Guess of 1.00e+00 produced a keff of 0.11843 +/- 0.00008
Iteration: 2; Guess of 5.00e+01 produced a keff of 2.12398 +/- 0.00074
Iteration: 3; Guess of 2.55e+01 produced a keff of 1.85429 +/- 0.00084
Iteration: 4; Guess of 1.32e+01 produced a keff of 1.35770 +/- 0.00068
Iteration: 5; Guess of 7.12e+00 produced a keff of 0.83458 +/- 0.00059
Iteration: 6; Guess of 1.02e+01 produced a keff of 1.12587 +/- 0.00078
Iteration: 7; Guess of 8.66e+00 produced a keff of 0.98785 +/- 0.00063
Iteration: 8; Guess of 9.42e+00 produced a keff of 1.05817 +/- 0.00068
Iteration: 9; Guess of 9.04e+00 produced a keff of 1.02340 +/- 0.00074
Iteration: 10; Guess of 8.85e+00 produced a keff of 1.00710 +/- 0.00073
Iteration: 11; Guess of 8.75e+00 produced a keff of 0.99754 +/- 0.00071
Iteration: 12; Guess of 8.80e+00 produced a keff of 1.00179 +/- 0.00058
Iteration: 13; Guess of 8.78e+00 produced a keff of 0.99913 +/- 0.00074
Iteration: 14; Guess of 8.79e+00 produced a keff of 1.00125 +/- 0.00069
I

### Mass error
in reference to PNNL HEU

In [21]:
error = np.empty(4)
error[0] = (BCMs[0]-BCMs[0])/BCMs[0]*100
error[1] = (BCMs[1]-BCMs[0])/BCMs[0]*100
error[2] = (BCMs[2]-BCMs[0])/BCMs[0]*100
error[3] = (BCMs[3]-BCMs[0])/BCMs[0]*100
print("Mass Error for the HEU24 approximation is %.5f percent" %error[1])
print("Mass Error for the Di-isotopic approximation is %.5f percent" %error[2])
print("Mass Error for the openmc enrichment approximation is %.5f percent" %error[3])


Mass Error for the HEU24 approximation is -0.19792 percent
Mass Error for the Di-isotopic approximation is 1.62597 percent
Mass Error for the openmc enrichment approximation is -0.06826 percent
